# King County Housing
#### House Price Estimate

**Authors:** Hatice Kastan, Czarina Luna, Ross McKim, Weston Shuken

##### January 2022

***

![image](Images/daria-nepriakhina-LZkbXfzJK4M-unsplash.jpg)

## Overview

    Overview of our project.

## Business Problem

    Stakeholder is a real estate company.
    Business Problem is predicting price and building a house price calculator.

## Data Understanding
    Describe the data being used for this project.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
raw_data = pd.read_csv('Data/kc_house_data.csv')

In [5]:
raw_data.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639


## Data Cleaning
    Describe and justify the process for preparing the data for analysis.

In [6]:
# Data prep and cleaning

# Change to datetime and add month column
raw_data['date'] = pd.to_datetime(raw_data['date'])
raw_data['month'] = pd.DatetimeIndex(raw_data['date']).month

# Change waterfront missing value to No
raw_data.loc[raw_data.waterfront.isnull(), 'waterfront'] = "NO"

# Change view missing value to None
raw_data.loc[raw_data.view.isnull(), 'view'] = "NONE"

# Change condition to numerical value
cond_dict = {'Poor':0, 'Fair':1, 'Average':2, 'Good':3, 'Very Good':4}
raw_data['condition'].replace(cond_dict, inplace=True)

# Change grade to numerical value
raw_data['grade'] = raw_data['grade'].map(lambda x: int(x.split(' ')[0]))

# Add has_basement column
raw_data['basement'] = raw_data['sqft_basement'].apply(lambda x: 0 if x == 0 else 1)

# Change some yr_renovated missing value to 0 and add renovated column
raw_data.loc[raw_data.yr_renovated.isnull(), 'yr_renovated'] = 0
raw_data['renovated'] = raw_data['yr_renovated'].apply(lambda x: 0 if x == 0 else 1)

# Add house_age column
raw_data['age'] = raw_data['date'].dt.year - raw_data['yr_built']

In [7]:
def corr_check(df, threshold):
    '''
    Enter dataframe and threshold for correlation
    Returns table of the highly correlated pairs
    '''
    corr_df = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    corr_df['pairs'] = list(zip(corr_df.level_0, corr_df.level_1))
    corr_df.set_index(['pairs'], inplace = True)
    corr_df.drop(columns=['level_1', 'level_0'], inplace = True)
    corr_df.columns = ['cc']
    corr_df = corr_df.drop_duplicates()
    corr_df = corr_df[(corr_df['cc'] > threshold) & (corr_df['cc'] < 1)]
    return corr_df

corr_check(raw_data, .7)

,cc
pairs,
"(renovated, yr_renovated)",0.999968
"(age, yr_built)",0.999873
"(sqft_living, sqft_above)",0.876448
"(sqft_living, grade)",0.762779
"(sqft_living, sqft_living15)",0.756402
"(sqft_above, grade)",0.756073
"(sqft_living, bathrooms)",0.755758
"(sqft_living15, sqft_above)",0.731767
"(sqft_lot, sqft_lot15)",0.718204


In [8]:
# Drop columns
raw_data.drop(columns=['id', 'yr_renovated', 'sqft_above', 'sqft_basement',
                      'yr_built', 'yr_renovated'], inplace=True)

In [9]:
raw_data.head(2)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,zipcode,lat,long,sqft_living15,sqft_lot15,month,basement,renovated,age
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NO,NONE,2,7,98178,47.5112,-122.257,1340,5650,10,1,0,59
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,2,7,98125,47.7210,-122.319,1690,7639,12,1,1,63


## Data Exploration
    Generate insights and visualizations about price and its relationships with variables.

In [10]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           21597 non-null  datetime64[ns]
 1   price          21597 non-null  float64       
 2   bedrooms       21597 non-null  int64         
 3   bathrooms      21597 non-null  float64       
 4   sqft_living    21597 non-null  int64         
 5   sqft_lot       21597 non-null  int64         
 6   floors         21597 non-null  float64       
 7   waterfront     21597 non-null  object        
 8   view           21597 non-null  object        
 9   condition      21597 non-null  int64         
 10  grade          21597 non-null  int64         
 11  zipcode        21597 non-null  int64         
 12  lat            21597 non-null  float64       
 13  long           21597 non-null  float64       
 14  sqft_living15  21597 non-null  int64         
 15  sqft_lot15     2159

In [11]:
raw_data.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,zipcode,lat,long,sqft_living15,sqft_lot15,month,basement,renovated,age
count,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.0,21597.000000,21597.000000
mean,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,2.409825,7.657915,98077.951845,47.560093,-122.213982,1986.620318,12758.283512,6.573969,1.0,0.034449,43.323286
std,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.650546,1.173200,53.513072,0.138552,0.140724,685.230472,27274.441950,3.115061,0.0,0.182384,29.377285
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,3.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,1.000000,1.0,0.000000,-1.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,2.000000,7.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000,4.000000,1.0,0.000000,18.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,2.000000,7.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000,6.000000,1.0,0.000000,40.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,3.000000,8.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,9.000000,1.0,0.000000,63.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,4.000000,13.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,12.000000,1.0,1.000000,115.000000


In [13]:
import statsmodels.api as sm

### Baseline Model
    Run simple linear regression on feature highest correlated with price.

In [14]:
# ols
mod1 = sm.formula.ols(formula= "price ~ sqft_living", data = raw_data)
mod1

In [15]:
mod1_result = mod1.fit()
mod1_result

In [16]:
mod1_summ = mod1_result.summary()
mod1_summ
#trying linear regression with one variable

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 2.097e+04
Date:                Mon, 03 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:43:51   Log-Likelihood:            -3.0006e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21595   BIC:                         6.001e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.399e+04   4410.023     -9.975      0.000   -5.26e+04   -3.53e+04
sqft_living   280.8630      1.939    144.819      0.000     277.062     284.664
==============================================================================
Omnibus:                    14801.942   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           542662.604
Skew:                           2.820   Prob(JB):                         0.00
Kurtosis:                      26.901   Cond. No.                     5.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
all_mod_summ = sm.formula.ols(formula='price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + waterfront + view + condition+ grade + month + basement + renovated + age', data= raw_data)
all_mod_summ

In [24]:
all_mod_result = all_mod_summ.fit()
all_mod_result

In [25]:
all_mod_summ = all_mod_result.summary()
all_mod_summ

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     2736.
Date:                Mon, 03 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:54:24   Log-Likelihood:            -2.9589e+05
No. Observations:               21597   AIC:                         5.918e+05
Df Residuals:                   21581   BIC:                         5.919e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -4.617e+05   9320.737    -49.534      0.000    -4.8e+05   -4.43e+05
waterfront[T.YES]  5.224e+05   2.19e+04     23.906      0.000     4.8e+05    5.65e+05
view[T.EXCELLENT]   2.37e+05   1.63e+04     14.519      0.000    2.05e+05    2.69e+05
view[T.FAIR]       5.802e+04   1.38e+04      4.209      0.000     3.1e+04     8.5e+04
view[T.GOOD]       6.086e+04   1.19e+04      5.123      0.000    3.76e+04    8.41e+04
view[T.NONE]      -5.249e+04   7301.375     -7.188      0.000   -6.68e+04   -3.82e+04
bedrooms          -3.886e+04   2030.829    -19.135      0.000   -4.28e+04   -3.49e+04
bathrooms          4.659e+04   3438.300     13.549      0.000    3.98e+04    5.33e+04
sqft_living         169.2426      3.280     51.604      0.000     162.814     175.671
sqft_lot             -0.2470      0.036     -6.798      0.000      -0.318      -0.176
floors             2.512e+04   3421.743      7.342      0.000    1.84e+04    3.18e+04
condition          1.958e+04   2482.157      7.887      0.000    1.47e+04    2.44e+04
grade              1.253e+05   2138.994     58.592      0.000    1.21e+05     1.3e+05
month             -2327.7585    471.670     -4.935      0.000   -3252.266   -1403.251
basement          -4.617e+05   9320.737    -49.534      0.000    -4.8e+05   -4.43e+05
renovated           2.35e+04   8488.234      2.769      0.006    6862.727    4.01e+04
age                3598.9518     69.620     51.694      0.000    3462.491    3735.413
==============================================================================
Omnibus:                    16031.403   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1118846.061
Skew:                           2.959   Prob(JB):                         0.00
Kurtosis:                      37.761   Cond. No.                     6.16e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.11e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

##### Model Metrics Table
    Create table of metrics we care about, and update with every additional model after.

In [11]:
# metric_df

## Feature Engineering
    Create new variables to predict the price.

In [12]:
# code

### Feature Scaling
    Perform log transformation and standardization.

In [13]:
# StandardScaler and PowerTransformer

### Feature Selection
    Feature ranking with recursive feature elimination.

In [14]:
# RFE

## Data Modeling
    Describe and justify the process for modeling the data.
    Run multiple linear regression on top ranking features.

In [15]:
# OLS

#### Check Assumptions of Linear Regression
    Linearity, independence, normality, homoescadicity

In [16]:
# code